# **Tarefa 4**: Álgebra Linear e Otimização para ML -  MO431A
Universidade Estadual de Campinas (UNICAMP), Instituto de Computação (IC)

Prof. Jacques Wainer, 2021s1


In [1]:
# RA & Name
print('265673: ' + 'Gabriel Luciano Gomes')
print('192880: ' + 'Lucas Borges Rondon')
print('265674: ' + 'Paulo Júnio Reis Rodrigues')

265673: Gabriel Luciano Gomes
192880: Lucas Borges Rondon
265674: Paulo Júnio Reis Rodrigues


## Imports necessários para a tarefa

In [2]:
import numpy as np

from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score, KFold, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_squared_error

from scipy.stats import loguniform, uniform

from hyperopt import hp, tpe, fmin, STATUS_OK

from pyswarm import pso

from cma import CMAEvolutionStrategy

from simanneal import Annealer

## Leitura da base de dados

In [3]:
X = np.load('db/X.npy')
Y = np.load('db/y.npy')

## Variáveis Globais

In [4]:
# C lower and upper bounds
c_lb = -5
c_ub = 15

# C lower and upper bounds
g_lb = -15
g_ub = 3

#epsilon lower and upper bounds
e_lb = 0.05
e_ub = 1.0

# Base SVM model
base_model = SVR(kernel = 'rbf')

## Funções úteis

### Computar RMSE

In [5]:
def compute_rmse(scores):
    # Compute RMSE
    return np.sqrt(np.mean(np.absolute(scores)))  

### Calcular cross val score

In [6]:
def hyperopt_train_test(params): 
    ''' Computes the cross validation score
    to be compared in order to identify
    the best value
    @params: list of params to SVR (C, gamma, epsilon and Kernel)
    '''
    clf = SVR(**params)
    return cross_val_score(clf, X, Y).mean()

### SVM Regressor

In [7]:
def compute_SVM_result(c, gamma, epsilon):    
    # define cross validation score
    cv = KFold(n_splits = 5, random_state = 1, shuffle = True)

    # Compute SVM
    svr = SVR(kernel = 'rbf', C = c, gamma = gamma, epsilon = epsilon)

    # SVM scores
    scores = cross_val_score(svr, X, Y, scoring = ('neg_mean_squared_error'), cv = cv) 
    
    show_results(c, gamma, epsilon, compute_rmse(scores))
    

### Exibir resultados

In [8]:
def show_results(c, gamma, epsilon, rmse) :
    print('----- Best values of hyperparameters ----- \n' +
      f'C: {round(c, 6)}\ngamma: {round(gamma, 6)} \nepsilon: {round(epsilon, 6)} \n' +
      '----- RMSE for given values ----- \n' + 
      f'RMSE: {round(rmse, 6)}')

## Random Search

In [9]:
# Search space
space = dict()
space['C'] = loguniform(2**c_lb, 2**c_ub)
space['gamma'] = loguniform(2**g_lb, 2**g_ub)
space['epsilon'] = uniform(e_lb, e_ub)

# define search
search = RandomizedSearchCV(base_model,
                            space,
                            n_iter = 125,
                            scoring = 'neg_mean_squared_error',
                            n_jobs = -1,
                            cv = 5,
                            random_state = 1)
result = search.fit(X, Y)
c = result.best_params_['C']
g = result.best_params_['gamma']
e = result.best_params_['epsilon']

### Resultados obtidos

In [10]:
compute_SVM_result(c, g, e)

----- Best values of hyperparameters ----- 
C: 8584.928547
gamma: 3.2e-05 
epsilon: 0.623679 
----- RMSE for given values ----- 
RMSE: 4.023489


## Grid Search

In [11]:
# grid size
g_size = 5

# Search space
space = dict()
space['C'] = loguniform.rvs(2**c_lb, 2**c_ub, size = g_size) 
space['gamma'] = loguniform.rvs(2**g_lb, 2**g_ub, size = g_size)
space['epsilon'] = uniform.rvs(e_lb, e_ub, size = g_size)

# define search
search = GridSearchCV(base_model,
                      space,
                      scoring = 'neg_mean_squared_error',
                      n_jobs = -1,
                      cv = 5)

result = search.fit(X, Y)
c = result.best_params_['C']
e = result.best_params_['epsilon']
g = result.best_params_['gamma']

### Resultados obtidos

In [12]:
compute_SVM_result(c, g, e)

----- Best values of hyperparameters ----- 
C: 87.143273
gamma: 0.000423 
epsilon: 1.002995 
----- RMSE for given values ----- 
RMSE: 5.22651


## Bayesian Optimization

In [13]:
def objective_function_bo(params):
    ''' Callable function to compare SVR scores.
    For this example, loss will be used.
    @params: list of params to SVR (C, gamma, epsilon and Kernel)
    '''   
    C =  params['C']
    gamma = params['gamma']
    epsilon = params['epsilon']
    acc = hyperopt_train_test({'C': 2**C, 'gamma': 2**gamma, 'epsilon': epsilon})
    
    return {'loss': -acc, 'status': STATUS_OK}

In [14]:
space = {
    'C': hp.uniform('C', c_lb, c_ub),
    'gamma': hp.uniform('gamma', g_lb, g_ub),
    'epsilon': hp.uniform('epsilon', e_lb, e_ub)   
}

best = fmin(objective_function_bo, space, algo = tpe.suggest, max_evals = 125)
c = 2** best['C']
g = 2** best['gamma']
e = best['epsilon']

100%|█████████████████████████████████████████████| 125/125 [03:52<00:00,  1.86s/trial, best loss: -0.8247652336636048]


### Resultados obtidos

In [15]:
compute_SVM_result(c, g, e)

----- Best values of hyperparameters ----- 
C: 14637.801447
gamma: 3.1e-05 
epsilon: 0.579594 
----- RMSE for given values ----- 
RMSE: 3.991933


## PSO

In [16]:
def objective_function_pso(x):
    C, gamma, epsilon = x
    kernel =  'rbf'
    acc = hyperopt_train_test({'C': 2**C, 'gamma': 2**gamma, 'epsilon': epsilon, 'kernel': kernel})
    return -acc

In [17]:
# upper and lower bounds for C, gamma and epsilon respectively
lb = [c_lb, g_lb, e_lb]
ub = [c_ub, g_ub, e_ub]

xopt, fopt = pso(objective_function_pso, lb, ub, swarmsize = 11, maxiter = 11)

c = 2** xopt[0]
g = 2** xopt[1]
e = xopt[2]

Stopping search: maximum iterations reached --> 11


### Resultados obtidos

In [18]:
compute_SVM_result(c, g, e)

----- Best values of hyperparameters ----- 
C: 17082.149431
gamma: 3.1e-05 
epsilon: 0.775143 
----- RMSE for given values ----- 
RMSE: 3.981901


## Simulated Annealing

Classe Filha do Annealing, necessária para funcionamento

In [19]:
class SimulatedAnnealing(Annealer):
    """Test annealer to objetctive function"""
    
    def __init__(self, state):
        super(SimulatedAnnealing, self).__init__(state)
    
    def move(self):
        """Swaps params of SVM."""
        self.state[0] = 2 ** np.random.uniform(low = c_lb, high = c_ub)
        self.state[1] = 2 ** np.random.uniform(low = g_lb, high = g_ub)
        self.state[2] = np.random.uniform(low = e_lb, high = e_ub)
        
    def energy(self):
        """Calculates cross validation score"""
        C, gamma, epsilon = self.state[0], self.state[1], self.state[2]
        kernel =  'rbf'
        
        return self.objective_function_sa({
            'C': C,
            'gamma': gamma,
            'epsilon': epsilon,
            'kernel': kernel
        })
    
    def objective_function_sa(self, x):        
        acc = hyperopt_train_test(x)        
        return -acc

In [20]:
initial_state = [
      2 ** np.random.uniform(low = c_lb, high = c_ub),
      2 ** np.random.uniform(low = g_lb, high = g_ub),
      np.random.uniform(low = e_lb, high = e_ub)    
]

sa = SimulatedAnnealing(initial_state)
sa.steps = 125

xopt, fopt = sa.anneal()
c = xopt[0]
g = xopt[1]
e = xopt[2]

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000          0.03   100.00%     0.00%     0:02:49     0:00:00

### Resultados obtidos

In [21]:
compute_SVM_result(c, g, e)

----- Best values of hyperparameters ----- 
C: 2718.172374
gamma: 9.8e-05 
epsilon: 0.121457 
----- RMSE for given values ----- 
RMSE: 4.31529


## CMA-ES

In [22]:
def objetive_function_CMA_ES(x):
    C, gamma, epsilon = x
    kernel =  'rbf'
    
    # compute cross val score with normalized hyperparams
    acc = hyperopt_train_test({'C': 2** (c_lb + C*20),
                               'gamma': 2** (g_lb + gamma*18),
                               'epsilon': e_lb + epsilon*0.95 ,
                               'kernel': kernel})
    return -acc

In [24]:
# Define initial bounds
lw = [0.0, 0.0, 0.0]
up = [1.0, 1.0, 1.0]

# Initial values
x0 = 3 * [0.05]
sigma = 0.25

result = CMAEvolutionStrategy(x0, sigma, {'bounds': [lw, up]})
result.optimize(objetive_function_CMA_ES, iterations = 125)

# extract best hyperparameters values normalizing it
c = 2 ** (c_lb + result.best.x[0] * 20)
g = 2 ** (g_lb + result.best.x[1] * 18)
e = e_lb + result.best.x[2]*0.95

(3_w,7)-aCMA-ES (mu_w=2.3,w_1=58%) in dimension 3 (seed=747261, Wed Apr 28 11:55:18 2021)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      7 -3.161849215501714e-01 1.0e+00 2.42e-01  2e-01  3e-01 0:01.5
    2     14 -5.906189742695303e-01 1.3e+00 2.79e-01  2e-01  4e-01 0:03.4
    3     21 -6.706682171259113e-01 1.7e+00 2.94e-01  2e-01  4e-01 0:06.0
    4     28 -8.213829486441921e-01 2.3e+00 3.55e-01  3e-01  6e-01 0:34.8
    5     35 -6.780610386013299e-01 2.6e+00 4.27e-01  3e-01  7e-01 1:17.9
    6     42 -7.402443128022795e-01 3.0e+00 4.16e-01  3e-01  7e-01 1:36.5
    7     49 -6.895381527236459e-01 2.5e+00 3.70e-01  3e-01  5e-01 1:56.7
    9     63 -6.429845008328915e-01 2.6e+00 3.16e-01  2e-01  4e-01 3:22.8
   10     70 -7.416259106937801e-01 2.6e+00 3.14e-01  2e-01  3e-01 4:06.6
   12     84 -7.746931352354519e-01 1.9e+00 2.32e-01  1e-01  2e-01 4:34.7
   13     91 -8.080529677968983e-01 2.0e+00 1.95e-01  1e-01  2e-01 5:54.0
   14     98 -8.13751275

### Resultados obtidos

In [25]:
compute_SVM_result(c, g, e)

----- Best values of hyperparameters ----- 
C: 22610.006226
gamma: 3.1e-05 
epsilon: 0.050666 
----- RMSE for given values ----- 
RMSE: 4.183407
